In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import math

Using TensorFlow backend.


In [29]:
weights_path = '/home/jose/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = "/home/jose/Masaüstü/ship_dataset_700_135/train"
validation_data_dir = "/home/jose/Masaüstü/ship_dataset_700_135/validation"
nb_train_samples = 3500
nb_validation_samples = 675
epochs = 50
batch_size = 16

In [38]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator,  int(math.ceil(nb_train_samples / batch_size)))
    np.save('bottleneck_features_train.npy',bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        generator, int(math.ceil(nb_validation_samples / batch_size)))
    
    np.save('bottleneck_features_validation.npy',bottleneck_features_validation)

In [93]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2)+ [2] * (nb_train_samples // 2)+ [3] * (nb_train_samples // 2)+ [4] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2) + [2] * (nb_validation_samples // 2)+ [3] * (nb_validation_samples // 2)+ [4] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  class_mode = "categorical",
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    model.fit(train_data, train_labels,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(validation_data, validation_labels))
    
    model.save_weights(top_model_weights_path)

In [40]:
save_bottlebeck_features()


Found 3500 images belonging to 5 classes.
Found 675 images belonging to 5 classes.


In [94]:
train_top_model()

ValueError: Error when checking target: expected dense_38 to have shape (5,) but got array with shape (1,)